## Equipe / Team

1. Afonso Henrique Torres Lucas | ifonso.developer@gmail.com | iFonso - 2215080047
2. Emanuelle Rocha Marreira
3. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
4. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
5. Ítalo Ferreira Fonseca
6. João Vitor Silva De Carvalho
7. Lilian Iazzai De Souza Oliveira | lilianiazzai@gmail.com | lilianiazzai - 2215080018
8. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

## Problem Description
- The quality of images generated by artificial intelligence has rapidly increased, raising concerns about authenticity and data security. This is due to the steady improvement of AI models capable of generating images that closely resemble what the human eye perceives as real. These artificially generated images are now competing with human-generated ones. In this context, CIFAKE is proposed as a dataset containing 60,000 synthetically generated images and 60,000 real images collected from CIFAR-10 (a commonly used low-resolution image dataset for machine learning).

- The images are of size 32x32 pixels, divided into 10 classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck.
Given this dataset, the task is to classify the images based on their nature—whether they are real or fake.

## Descrição do Problema

- A qualidade de imagens geradas por inteligência artificial tem rapidamente aumentado, levando a preocupação da autenticidade e da segurança dos dados levantados, isso se dá pelo aumento gradativo de melhores modelos de inteligência gerando fotos fiéis ao que o olho humano intepreta como real. Essas imagens geradas artifialmente estão no pódio a disputar com imagens geradas por humanos. Com isso, CIFAKE nos é proposto como um conjunto de dados que contém 60000 imagens sintéticamente geradas a 60000 imagens reais coletadas pelo CIFAR10 (conjunto de imagens utilizadas para aprendizado de máquina de baixa resolução).

- A qualidade das imagens apresentadas são do tamanho 32x32 pixels, divididas em 10 classes: avião, automóvel, pássaro, gato, viado, cachorro, sapo, cavalo, navio e caminhão.
Dado o dataset proposto fica a critério do modelo classificar as imagens de acordo com sua natureza, sendo real ou falso.

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    # Resize da imagem pra 32x32 (pixels)
    transforms.Resize((32, 32)),
    # PIL -> Tensor[canal, altura, largura] + Normalização de pixels
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root="../data/train", transform=transform)
test_data = datasets.ImageFolder(root="../data/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

Inicialmente, foi definida uma transformação para os dados carregados, redimensionando todas as imagens do dataset para 32×32 pixels. Em seguida, é aplicada outra transformação que converte as imagens do formato padrão PIL `[altura, largura, canais]` para o formato `[canais, altura, largura]` utilizado pelo PyTorch, além de normalizar os valores de pixel de 0–255 para o intervalo $[0.0, 1.0]$. No fim temos dataloaders de teste e de treino com batche de 32 para facilitar a iteração sobre o conjunto de dados.

Nosso conjunto de dados é composto por duas classes, de imagens reais e de imagens geradas artificialente, a seguir temos as classes:

In [3]:
print(train_data.classes)
print(train_data.class_to_idx)

['FAKE', 'REAL']
{'FAKE': 0, 'REAL': 1}


In [4]:
print(f"Dados:\n\ntreino: {len(train_data)} \nteste: {len(test_data)}")

Dados:

treino: 100000 
teste: 20000


In [5]:
fake_class_data_count = sum(1 for _, label in train_data if label == 1)
real_class_data_count = sum(1 for _, label in train_data if label == 0)

print(f"Quantidade de dados de treino:\nlabel 'REAL':{real_class_data_count} \nlabel 'FAKE':{fake_class_data_count}")

Quantidade de dados de treino:
label 'REAL':50000 
label 'FAKE':50000
